### Read dataset

In [2]:
import pandas as pd
import numpy as np
import os

df = pd.read_csv('../datasets/meals/DatasetMeals.csv')
df.head()
df['Category'] = df['Category'].str.replace('side', 'Side')
df = df.dropna(subset=['Meal_Ingredient_Ids'])
main_meals = df[df['Category'] == 'Main']
side_meals = df[ df['Category'] == 'Side']

main_meals.head()

,Meal_Id,Name,Meal_Ingredient_Ids,Meal_Ingredients_Names,Meal_Ingredient_Quantities,Category,Description,Supported_Main_Meals,Combinations_Supported,Max_Combinations,Min_Combinations,Morning_Probability,Lunch_Probability,Dinner_Probability
0,1,Kottu Roti,"SLI019,SLB002,SLD055,SLH006,SLK004,SLK002","Roti,carrot,leeks, Chicken,Coconut Oil,Chilli ...","30,20,27.5,20,5,2.5",Main,Sri Lankan street food,NaN,1,1.0,1.0,1.0,2.5,8.0
1,2,Hoppers,"SLI019,SLI005,SLI011,SLK004","Rice flour, Coconut milk, Sugar,Coconut Oil","70, 10, 5,15,",Main,Delicate rice flour pancakes,NaN,0,NaN,NaN,8.0,0.0,8.0
2,3,Kiribath,"SLA009,SLI005","Rice, Coconut milk","85,15",Main,Traditional coconut milk rice,NaN,1,1.0,1.0,9.0,1.0,0.0
6,8,Pittu,"SLI019,SLI005","Rice flour, Coconut","25,10",Main,Steamed coconut rice flour,NaN,1,1.0,1.0,8.0,6.0,4.0
7,9,Lamprais,"SLA008,SLH006,SLK006","Rice, Meat, Spices","100,30,10",Main,Dutch-influenced rice and curry,NaN,1,1.0,1.0,4.0,9.0,9.0


In [3]:
meal_combinations = []
side_meals['Supported_Main_Meals'] = side_meals['Supported_Main_Meals'].astype(str).fillna('')

def convert_to_int_list(cat_str):
    if not cat_str:
        return []
    return [int(x.strip()) for x in cat_str.split(',') if x.strip().isdigit()]

side_meals['Supported_Main_Meals_list'] = side_meals['Supported_Main_Meals'].apply(convert_to_int_list)

for _, main_meal in main_meals.iterrows():
    main_id = int(main_meal['Meal_Id'])
    main_name = main_meal['Name']
    related_side_meals = side_meals[side_meals['Supported_Main_Meals_list'].apply(lambda x: main_id in x)]
    for _, side_meal in related_side_meals.iterrows():
        side_name = side_meal['Name']
        meal_combinations.append({
            'Main Id': int(main_id),
            'Side Id':int(side_meal['Meal_Id']),
            'Main Meal': main_name,
            'Side Meal': side_name
        })

meal_combinations_df = pd.DataFrame(meal_combinations)

/var/folders/_4/xfkz0qss5jl85xqrndqzqrxr0000gn/T/ipykernel_1875/2590512694.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  side_meals['Supported_Main_Meals'] = side_meals['Supported_Main_Meals'].astype(str).fillna('')
/var/folders/_4/xfkz0qss5jl85xqrndqzqrxr0000gn/T/ipykernel_1875/2590512694.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  side_meals['Supported_Main_Meals_list'] = side_meals['Supported_Main_Meals'].apply(convert_to_int_list)


In [4]:
meal_combinations_df

,Main Id,Side Id,Main Meal,Side Meal
0,1,21,Kottu Roti,Chicken Curry
1,1,29,Kottu Roti,Devilled Chicken
2,1,30,Kottu Roti,Devilled Fish
3,1,89,Kottu Roti,Seer Fish Curry
4,3,4,Kiribath,Fish Ambul Thiyal
...,...,...,...,...
287,171,196,Roast Bread,Indian Fish Curry
288,198,21,Chicken Koththu,Chicken Curry
289,198,29,Chicken Koththu,Devilled Chicken
290,198,30,Chicken Koththu,Devilled Fish


In [5]:
import itertools
import pandas as pd
import random

combined_meals = []

################### For combinations supported main meals ########################
combinations_supported_meals = main_meals[main_meals['Combinations_Supported'] == 1]

for index, main_meal in combinations_supported_meals.iterrows():
    red_rice_combinations = meal_combinations_df[meal_combinations_df['Main Meal'] == main_meal["Name"]]
    side_meal_groups = red_rice_combinations.groupby('Main Meal')['Side Meal'].apply(list).reset_index()
    side_meals_for_red_rice = side_meal_groups.loc[side_meal_groups['Main Meal'] == main_meal["Name"], 'Side Meal'].values[0]

    def get_random_combinations(side_meals, number_of_meals, n_samples):
        all_combinations = list(itertools.combinations(side_meals, number_of_meals))
        sampled_combinations = random.sample(all_combinations, min(n_samples, len(all_combinations)))
        return sampled_combinations

    combinations = []

    for r in list(range(int(main_meal["Min_Combinations"]), int(main_meal["Max_Combinations"]) + 1)):
        sampled_combos = get_random_combinations(side_meals_for_red_rice, r, n_samples=10000)
        for combo in sampled_combos:
            combinations.append({
                'Main Meal': main_meal["Name"],
                'Side Meal': list(combo),
                'Breakfast_Probability': main_meal["Morning_Probability"],
                'Lunch_Probability': main_meal["Lunch_Probability"],
                'Dinner_Probability': main_meal["Dinner_Probability"],
                'Combined_Meal': 1,
            })

    combined_meals.extend(combinations)



################### For combinations NOT supported main meals ########################
combinations_not_supported_meals = main_meals[main_meals['Combinations_Supported'] == 0]
for index, main_meal in combinations_not_supported_meals.iterrows():
    combined_meals.append({
                'Main Meal': main_meal["Name"],
                'Side Meal': [],
                'Breakfast_Probability': main_meal["Morning_Probability"],
                'Lunch_Probability': main_meal["Lunch_Probability"],
                'Dinner_Probability': main_meal["Dinner_Probability"],
                'Combined_Meal': 0,
                'Main Meal': main_meal["Name"],
                'Main Meal': main_meal["Name"],
            })

combined_meal_combinations_df = pd.DataFrame(combined_meals)
combined_meal_combinations_df


,Main Meal,Side Meal,Breakfast_Probability,Lunch_Probability,Dinner_Probability,Combined_Meal
0,Kottu Roti,[Chicken Curry],1.0,2.5,8.0,1
1,Kottu Roti,[Devilled Chicken],1.0,2.5,8.0,1
2,Kottu Roti,[Devilled Fish],1.0,2.5,8.0,1
3,Kottu Roti,[Seer Fish Curry],1.0,2.5,8.0,1
4,Kiribath,[Seeni Sambol],9.0,1.0,0.0,1
...,...,...,...,...,...,...
10463,Pineapple Rice,[],5.0,5.0,5.0,0
10464,Prawn Tempered,[],7.0,7.0,7.0,0
10465,Sri lankan Pancakes,[],1.0,1.0,1.0,0
10466,Vegetable Lasagna,[],2.0,2.0,2.0,0


In [6]:
combined_meal_combinations_df_copy = combined_meal_combinations_df.copy()

combined_meal_combinations_df_copy['Side Meal'] = combined_meal_combinations_df['Side Meal'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

def create_complete_meal_name(row):
    if len(row['Side Meal']) > 0:
        return f"{row['Main Meal']} with {row['Side Meal']}"
    else:
        return row['Main Meal']

combined_meal_combinations_df_copy['Complete Meal'] = combined_meal_combinations_df_copy.apply(create_complete_meal_name, axis=1)
combined_meal_combinations_df_copy['Ingredients'] = ''
combined_meal_combinations_df_copy['Quantities'] = ''


for i in range(len(combined_meal_combinations_df_copy)):
    main_meal = combined_meal_combinations_df_copy.loc[i, 'Main Meal']
    side_meal = combined_meal_combinations_df_copy.loc[i, 'Side Meal']
    ingredient_dict_name = {}
    ingredient_dict = {}

    side_meal_list = side_meal.strip().split(', ')
    
    main_meal_ingredients = df[df['Name'] == main_meal]['Meal_Ingredients_Names'].values[0].split(',')
    # in main_meal_ingredients remove " " values
    # Remove empty string values
    main_meal_ingredients = [ingredient for ingredient in main_meal_ingredients if ingredient != " "]
    main_meal_quantities = df[df['Name'] == main_meal]['Meal_Ingredient_Quantities'].values[0].split(',')
    
    ingredient_id_list = df[df['Name'] == main_meal]['Meal_Ingredient_Ids'].values[0].split(',')
    ingredient_id_list = [ingredient for ingredient in ingredient_id_list if ingredient != ""]
    combined_ingredients_id = ingredient_id_list
    combined_quantities = main_meal_quantities
    combined_ingredients = main_meal_ingredients
    ingredient_dict = dict(zip(combined_ingredients_id, combined_quantities))
    ingredient_dict_name = dict(zip(combined_ingredients_id, combined_ingredients))
    
    for side_meal in side_meal_list:
        if side_meal.strip() == '':
            continue
        side_meal_ingredients = df[df['Name'] == side_meal]['Meal_Ingredients_Names'].values[0].split(',')
        side_meal_quantities = df[df['Name'] == side_meal]['Meal_Ingredient_Quantities'].values[0].split(',')
        side_meal_ingredients_id = df[df['Name'] == side_meal]['Meal_Ingredient_Ids'].values[0].split(',')
        
        
        for ingredient, quantity in zip(side_meal_ingredients_id , side_meal_quantities ):

            if ingredient in ingredient_dict:
                # Convert the current value to float before adding
                ingredient_dict[ingredient] = float(ingredient_dict[ingredient]) + float(quantity)
            else:
                ingredient_dict[ingredient] = float(quantity)
        
        for ingredient, name in zip(side_meal_ingredients_id , side_meal_ingredients):
            if ingredient in ingredient_dict_name:
                pass
            else:
                ingredient_dict_name[ingredient] = name
   
    combined_meal_combinations_df_copy.loc[i, 'Meal_Ingredient_Ids'] = ', '.join(list(ingredient_dict.keys()))
    combined_meal_combinations_df_copy.loc[i, 'Quantities'] = ', '.join([str(val) for val in list(ingredient_dict.values())])
    combined_meal_combinations_df_copy.loc[i, 'Ingredients'] = ', '.join([str(val) for val in list(ingredient_dict_name.values())])

combined_meal_combinations_df_copy.tail(20)


,Main Meal,Side Meal,Breakfast_Probability,Lunch_Probability,Dinner_Probability,Combined_Meal,Complete Meal,Ingredients,Quantities,Meal_Ingredient_Ids
10448,Roast Bread,Seer Fish Curry,3.0,3.0,3.0,1,Roast Bread with Seer Fish Curry,"Flour, sugar, Seer fish, Coconut milk, Chil...","50, 20, 50.0, 25.0, 10.0","SLA018, SLJ002, SLF025, SLI005, SLK002"
10449,Roast Bread,Simmered Fish Roe,3.0,3.0,3.0,1,Roast Bread with Simmered Fish Roe,"Flour, sugar, Fish, flour","80.0, 20, 70.0, 20.0","SLA018, SLJ002, SLF054, SLH001"
10450,Chicken Koththu,Ginger Chicken,1.0,2.5,8.0,1,Chicken Koththu with Ginger Chicken,"Flour, Chicken, Carrot, Egg, onion, Ginge...","80, 90.0, 20, 20, 10, 25.0, 10.0","SLA018, SLH006, SLB002, SLH001, SLD060, SLK012..."
10451,Chicken Koththu,Chicken Curry,1.0,2.5,8.0,1,Chicken Koththu with Chicken Curry,"Flour, Chicken, Carrot, Egg, onion, Cocon...","80, 90.0, 20, 20, 10, 20.0, 5.0, 3.0","SLA018, SLH006, SLB002, SLH001, SLD060, SLI00..."
10452,Chicken Koththu,Devilled Fish,1.0,2.5,8.0,1,Chicken Koththu with Devilled Fish,"Flour, Chicken, Carrot, Egg, onion, Fish, ...","80, 40, 20, 20, 30.0, 50.0, 5.0, 5.0","SLA018, SLH006, SLB002, SLH001, SLD060, SLF025..."
10453,Chicken Koththu,Devilled Chicken,1.0,2.5,8.0,1,Chicken Koththu with Devilled Chicken,"Flour, Chicken, Carrot, Egg, onion, Chili...","80, 90.0, 20, 20, 30.0, 5.0, 10.0","SLA018, SLH006, SLB002, SLH001, SLD060, SLK002..."
10454,Hoppers,,8.0,0.0,8.0,0,Hoppers,"Rice flour, Coconut milk, Sugar, Coconut Oil","70, 10, 5, 15","SLI019, SLI005, SLI011, SLK004"
10455,Pumpkin Soup,,8.0,1.0,2.0,0,Pumpkin Soup,"Pumpkin, Coconut milk, Spices","50, 25, 10","SLD071, SLI005, SLK006"
10456,Vegetable Soup,,5.0,5.0,5.0,0,Vegetable Soup,"Carrot, Cabbage, Pumpkin, Mushroom","30, 40, 30, 10","SLB002, SLD021, SLD071, SLD059"
10457,Milk Rice,,5.0,5.0,5.0,0,Milk Rice,"Rice, Coconut milk","50, 25, 10","SLA008, SLI005, SLK002"


In [7]:
combined_meal_combinations_df_copy[combined_meal_combinations_df_copy["Main Meal"] == "Hoppers"]

,Main Meal,Side Meal,Breakfast_Probability,Lunch_Probability,Dinner_Probability,Combined_Meal,Complete Meal,Ingredients,Quantities,Meal_Ingredient_Ids
10454,Hoppers,,8.0,0.0,8.0,0,Hoppers,"Rice flour, Coconut milk, Sugar, Coconut Oil","70, 10, 5, 15","SLI019, SLI005, SLI011, SLK004"


In [8]:
df_nut = pd.read_csv('../datasets/ingredients/final_ingredients.csv', index_col="Food_Code")
df_nut.head()

df_nut = df_nut.replace(['Na','ND','-'], np.nan)
df_nut.fillna(0, inplace=True)

num = 0
def check_validity(row):
    global num 
    ingredient_ids = row['Meal_Ingredient_Ids'].split(',')
    ingredients = row['Meal_Ingredients_Names'].split(',')
    if len(ingredient_ids) != len(ingredients):
        num = num + 1

df.apply(check_validity, axis=1)
print(f"Total errors: {num}")

nutritional_columns = [
    'Energy(Kcal)', 'Protein(g)', 'Total fat(g)', 'Carbohydrates(g)', 
    'Total Dietary Fibre(g)', 'Vitamin A(µg)', 'Vitamin D(µg)', 'Vitamin K(µg)', 
    'Vitamin E(mg)', 'Calcium(mg)', 'Phosphorus(mg)', 'Magnesium(mg)', 
    'Sodium(mg)', 'Potassium(mg)', 'Saturated Fatty Acids(mg)', 
    'Monounsaturated Fatty Acids(mg)', 'Polyunsaturated Fatty Acids(mg)', 
    'Free sugar(g)', 'Starch(g)'
]

df_nut['Total Dietary Fibre(g)'] = df_nut['Total Dietary Fibre(g)'].astype('float64')
df_nut['Vitamin A(µg)'] = df_nut['Vitamin A(µg)'].astype('float64')
df_nut['Vitamin D(µg)'] = df_nut['Vitamin D(µg)'].astype('float64')
df_nut['Vitamin K(µg)'] = df_nut['Vitamin K(µg)'].astype('float64')
df_nut['Vitamin E(mg)'] = df_nut['Vitamin E(mg)'].astype('float64')
df_nut['Saturated Fatty Acids(mg)'] = df_nut['Saturated Fatty Acids(mg)'].astype('float64')
df_nut['Monounsaturated Fatty Acids(mg)'] = df_nut['Monounsaturated Fatty Acids(mg)'].astype('float64')
df_nut['Polyunsaturated Fatty Acids(mg)'] = df_nut['Polyunsaturated Fatty Acids(mg)'].astype('float64')

_errored_ingredients = []

def calculate_total_nutrition(row):
    ingredient_ids = row['Meal_Ingredient_Ids'].split(',')
    quantities = list(map(float, row['Quantities'].split(',')))

    total_nutrition = {col: 0 for col in nutritional_columns}

    
    i = 0
    for ingredient_id in ingredient_ids:
        ingredient_id = ingredient_id.strip()
        if ingredient_id == '':
            i += 1
            continue
        
        try:
            nutrition_values = df_nut.loc[ingredient_id]
            
            for nutrient in total_nutrition.keys():
                value_nutrient = 0
                try:
                    value_nutrient = float(nutrition_values[nutrient])
                except:
                    value_nutrient = 0
                
                total_nutrition[nutrient] += (value_nutrient * quantities[i]) / 100
        except:
            if ingredient_id not in _errored_ingredients:
                _errored_ingredients.append(ingredient_id)

        i += 1

    
    return pd.Series(total_nutrition)
nutritional_values_df = combined_meal_combinations_df_copy.apply(calculate_total_nutrition, axis=1)
df_combined_meal_with_nutrition = pd.concat([combined_meal_combinations_df_copy, nutritional_values_df], axis=1)


print("Please check these ingredients....")
print(_errored_ingredients)





Total errors: 12
Please check these ingredients....
['SLH018']


In [9]:
df_combined_meal_with_nutrition

,Main Meal,Side Meal,Breakfast_Probability,Lunch_Probability,Dinner_Probability,Combined_Meal,Complete Meal,Ingredients,Quantities,Meal_Ingredient_Ids,...,Calcium(mg),Phosphorus(mg),Magnesium(mg),Sodium(mg),Potassium(mg),Saturated Fatty Acids(mg),Monounsaturated Fatty Acids(mg),Polyunsaturated Fatty Acids(mg),Free sugar(g),Starch(g)
0,Kottu Roti,Chicken Curry,1.0,2.5,8.0,1,Kottu Roti with Chicken Curry,"Roti, carrot, leeks, Chicken, Coconut Oil, Ch...","30, 20, 27.5, 70.0, 5, 2.5, 20.0, 5.0, 3.0","SLI019, SLB002, SLD055, SLH006, SLK004, SLK002...",...,90.11225,242.7032,68.4165,41.98655,584.061,9412.309,2063.6709,835.6485,2.31535,24.0562
1,Kottu Roti,Devilled Chicken,1.0,2.5,8.0,1,Kottu Roti with Devilled Chicken,"Roti, carrot, leeks, Chicken, Coconut Oil, Ch...","30, 20, 27.5, 70.0, 5, 7.5, 10.0, 20.0","SLI019, SLB002, SLD055, SLH006, SLK004, SLK002...",...,102.76675,202.0175,63.7480,41.89725,518.101,801.731,804.1420,616.0965,1.09825,24.3140
2,Kottu Roti,Devilled Fish,1.0,2.5,8.0,1,Kottu Roti with Devilled Fish,"Roti, carrot, leeks, Chicken, Coconut Oil, Ch...","30, 20, 27.5, 20, 5, 7.5, 50.0, 5.0, 20.0","SLI019, SLB002, SLD055, SLH006, SLK004, SLK002...",...,66.16175,261.2955,62.8230,43.12925,573.501,1514.681,848.9505,1103.7465,1.06725,24.1670
3,Kottu Roti,Seer Fish Curry,1.0,2.5,8.0,1,Kottu Roti with Seer Fish Curry,"Roti, carrot, leeks, Chicken, Coconut Oil, Ch...","30, 20, 27.5, 20, 5, 12.5, 50.0, 25.0","SLI019, SLB002, SLD055, SLH006, SLK004, SLK002...",...,34.63225,304.5530,60.5835,43.05575,650.951,12247.829,1762.4700,1318.2885,2.59775,23.8960
4,Kiribath,Seeni Sambol,9.0,1.0,0.0,1,Kiribath with Seeni Sambol,"Rice, Coconut milk, Onion, Sugar, chillii","85, 15, 50.0, 5.0, 5.0","SLA009, SLI005, SLD060, SLI011, SLK002",...,27.73800,215.7965,78.3285,12.79200,440.200,6976.655,1049.0970,779.0950,6.40750,64.8125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10463,Pineapple Rice,,5.0,5.0,5.0,0,Pineapple Rice,"Pineapple, Rice","30, 80","SLE041, SLA012",...,10.07900,156.8460,33.0210,2.14500,163.100,553.311,491.2910,653.9440,2.69300,61.7040
10464,Prawn Tempered,,7.0,7.0,7.0,0,Prawn Tempered,"Prawns, Oil","90, 20","SLF043, SLJ002",...,34.02900,170.1000,41.3370,169.20000,126.900,127.116,70.4030,91.1540,0.00000,0.0000
10465,Sri lankan Pancakes,,1.0,1.0,1.0,0,Sri lankan Pancakes,"Flour, Egg, coconut, sugar","90, 50, 30, 20","SLA018, SLH001, SLI005, SLI011",...,59.37200,254.7000,67.2660,85.40300,512.700,14892.090,3675.3000,1437.2000,23.57600,64.0080
10466,Vegetable Lasagna,,2.0,2.0,2.0,0,Vegetable Lasagna,"Carrot, Cabbage, Pasta, Cheese","30, 20, 70, 20","SLB002, SLD021, SLA018, SLG002",...,167.33300,202.0990,35.0720,281.79000,329.600,99.297,46.0020,335.0020,2.33400,50.3500


In [10]:
df_prices = pd.read_csv('../datasets/ingredients/final_prices.csv', index_col="Price_ID")
df_prices.head()

df_prices = df_prices.replace(['Na','ND','-'], np.nan)
df_prices.fillna(0, inplace=True)

_errored_ingredients = []

def calculate_total_price(row):
    ingredient_ids = row['Meal_Ingredient_Ids'].split(',')
    quantities = list(map(float, row['Quantities'].split(',')))
    price = 0
    i = 0
    for ingredient_id in ingredient_ids:
        ingredient_id = ingredient_id.strip()
        if ingredient_id == '':
            i += 1
            continue
        
        try:
            nutrition_row = df_nut.loc[ingredient_id]
            price_row = df_prices.loc[nutrition_row["Price_Code"]]

            price += (float(price_row["Price_Per_100_Unit"]) * quantities[i]) / 100
        except:
            if ingredient_id not in _errored_ingredients:
                _errored_ingredients.append(ingredient_id)

        i += 1
    
    return price
df_combined_meal_with_nutrition["Price"] = df_combined_meal_with_nutrition.apply(calculate_total_price, axis=1)


print("Please check these ingredients....")
print(_errored_ingredients)

df_combined_meal_with_nutrition.to_csv("../datasets/meals/FinalMeals.csv", index=False)


Please check these ingredients....
['SLI005', 'SLF045', 'SLH018']


In [282]:
meal_counts = df_combined_meal_with_nutrition.groupby('Main Meal').size()
df_combined_meal_with_nutrition_copy = df_combined_meal_with_nutrition.copy()

df_combined_meal_with_nutrition_copy.set_index('Complete Meal',inplace=True)
df_combined_meal_with_nutrition_copy = df_combined_meal_with_nutrition_copy[~df_combined_meal_with_nutrition_copy.index.duplicated(keep='first')]

def adjust_probability(row, column):
    if row['Combined_Meal'] == 1:
        similar_meal_count = meal_counts.get(row['Main Meal'], 1)
        return row[column] / (similar_meal_count ** 0.62)
    return row[column]

nutrition_types = ['Energy(Kcal)', 'Protein(g)', 'Total fat(g)', 'Carbohydrates(g)', 'Magnesium(mg)', 
              'Sodium(mg)', 'Potassium(mg)', 'Saturated Fatty Acids(mg)', 
              'Monounsaturated Fatty Acids(mg)', 'Polyunsaturated Fatty Acids(mg)', 
              'Free sugar(g)', 'Starch(g)']

def get_random_meals(num_iterations=1):
    df_combined_meal_with_nutrition["True_Breakfast_Probability"] =  df_combined_meal_with_nutrition.apply(lambda row: adjust_probability(row, 'Breakfast_Probability'), axis=1)
    df_combined_meal_with_nutrition["True_Lunch_Probability"] =  df_combined_meal_with_nutrition.apply(lambda row: adjust_probability(row, 'Lunch_Probability'), axis=1)
    df_combined_meal_with_nutrition["True_Dinner_Probability"] =  df_combined_meal_with_nutrition.apply(lambda row: adjust_probability(row, 'Dinner_Probability'), axis=1)
    
    meal_permutations = []
    
    for _ in range(num_iterations):
        breakfast = random.choices(
            df_combined_meal_with_nutrition['Complete Meal'], 
            weights=df_combined_meal_with_nutrition['True_Breakfast_Probability'], 
            k=1
        )[0]

        lunch = random.choices(
            df_combined_meal_with_nutrition['Complete Meal'], 
            weights=df_combined_meal_with_nutrition['True_Lunch_Probability'],
            k=1
        )[0]

        dinner = random.choices(
            df_combined_meal_with_nutrition['Complete Meal'], 
            weights=df_combined_meal_with_nutrition['True_Dinner_Probability'],
            k=1
        )[0]

        breakfast_row = df_combined_meal_with_nutrition_copy.loc[breakfast]
        lunch_row = df_combined_meal_with_nutrition_copy.loc[lunch]
        dinner_row = df_combined_meal_with_nutrition_copy.loc[dinner]

        total_nutrition = {}
        for nutrient in nutrition_types:
            total_key = f'Total {nutrient}'
            total_nutrition[total_key] = float(breakfast_row[nutrient]) + float(lunch_row[nutrient]) + float(dinner_row[nutrient])
        
        total_price =  float(breakfast_row["Price"]) + float(lunch_row["Price"]) + float(dinner_row["Price"])

        meal_permutations.append({
            'Breakfast': breakfast,
            'Lunch': lunch,
            'Dinner': dinner,
            'Price': total_price,
            **total_nutrition
        })

    return meal_permutations

preference_permutations_df = pd.DataFrame(
    get_random_meals(num_iterations=200000)
)

preference_permutations_df


,Breakfast,Lunch,Dinner,Price,Total Energy(Kcal),Total Protein(g),Total fat(g),Total Carbohydrates(g),Total Magnesium(mg),Total Sodium(mg),Total Potassium(mg),Total Saturated Fatty Acids(mg),Total Monounsaturated Fatty Acids(mg),Total Polyunsaturated Fatty Acids(mg),Total Free sugar(g),Total Starch(g)
0,"Red Rice with Fish Curry, Tapioca Chips, Batu ...","Red Rice with Banana Blossom Curry, Green Gram...",Vegetable Kottu with Devilled Fish,792.300000,1482.370,73.53570,83.15660,175.93710,478.5000,128.36700,3346.980,37105.98150,6314.27410,5152.7615,11.74420,151.10370
1,"Red Rice with Parippu, Brinjal Moju, Ginger Ch...",Kottu Roti with Chicken Curry,"Red Rice with Fish Curry, Kiri Hodi, Sprats Fry",748.878571,1403.335,72.67590,116.79305,113.99815,372.0280,149.75905,2679.591,50047.41900,10200.68900,3867.1725,10.18755,94.45990
2,"Red Rice with Coriander Sambol, Coconut Sambol...","Red Rice with Chickpea Curry, Fenugreek Curry,...","Vegetable Biryani with Chicken Curry, Devilled...",755.200000,2006.658,95.27754,154.73754,213.91446,681.0470,216.68126,4178.252,81049.98900,18479.57398,8075.9740,19.29102,166.11654
3,Hoppers,"Red Rice with Elumas Curry, Potato Curry, Devi...",Pumpkin Soup,408.125000,1273.780,39.33400,111.29760,107.31420,242.9905,72.48640,1663.584,39954.54900,8905.82360,2540.5365,12.37140,88.51040
4,"Red Rice with Bathala Curry, Mustard Greens Cu...",String Hoppers with Pol Sambol,Coconut Roti with Dhal Curry,304.678000,1378.880,57.99280,93.03370,202.32750,308.9966,99.57520,2477.070,60123.21380,7188.46100,3299.2409,13.44240,172.60020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,Fish Kottu with Devilled Chicken,"Egg Fried Rice with Ala Badun, Vegetable Cutlets","Red Rice with Chicken Curry, Cashew Curry, Tro...",909.971000,1734.480,73.59915,82.75265,212.15410,463.6570,204.72410,3102.770,27029.65475,20864.31980,7472.8970,14.85395,182.24390
199996,"Red Rice with Parippu, Tomato Curry, Vegetable...","Chicken Fried Rice with Ala Badun, Cashew Curry",Coconut Roti with Fish Ambul Thiyal,1082.542000,1775.250,70.30400,116.97850,192.51750,500.0985,138.20800,3251.318,46516.95900,21770.17700,8176.2990,11.33950,165.67700
199997,Hoppers,"Red Rice with Chicken Curry, Beetroot curry, K...","Vegetable Fried Rice with Wambatu Moju, Cashew...",611.507857,1755.605,53.40785,117.71200,223.42295,405.8020,121.73010,2307.994,52973.73200,20356.11515,6584.9755,18.49190,172.29110
199998,Gotu Kola Kanda,"Red Rice with Pineapple Curry, Beetroot curry,...","Egg Fried Rice with Ala Badun, Fish Cutlets",483.850000,1384.290,43.96545,91.19495,164.94830,337.0390,213.43830,2599.690,35868.61975,7705.21390,3619.7880,14.26885,140.10920


In [284]:
preference_permutations_df.to_csv("../datasets/meals/FinalPermutations.csv", index=False)


In [286]:
preference_permutations_df

,Breakfast,Lunch,Dinner,Price,Total Energy(Kcal),Total Protein(g),Total fat(g),Total Carbohydrates(g),Total Magnesium(mg),Total Sodium(mg),Total Potassium(mg),Total Saturated Fatty Acids(mg),Total Monounsaturated Fatty Acids(mg),Total Polyunsaturated Fatty Acids(mg),Total Free sugar(g),Total Starch(g)
0,"Red Rice with Fish Curry, Tapioca Chips, Batu ...","Red Rice with Banana Blossom Curry, Green Gram...",Vegetable Kottu with Devilled Fish,792.300000,1482.370,73.53570,83.15660,175.93710,478.5000,128.36700,3346.980,37105.98150,6314.27410,5152.7615,11.74420,151.10370
1,"Red Rice with Parippu, Brinjal Moju, Ginger Ch...",Kottu Roti with Chicken Curry,"Red Rice with Fish Curry, Kiri Hodi, Sprats Fry",748.878571,1403.335,72.67590,116.79305,113.99815,372.0280,149.75905,2679.591,50047.41900,10200.68900,3867.1725,10.18755,94.45990
2,"Red Rice with Coriander Sambol, Coconut Sambol...","Red Rice with Chickpea Curry, Fenugreek Curry,...","Vegetable Biryani with Chicken Curry, Devilled...",755.200000,2006.658,95.27754,154.73754,213.91446,681.0470,216.68126,4178.252,81049.98900,18479.57398,8075.9740,19.29102,166.11654
3,Hoppers,"Red Rice with Elumas Curry, Potato Curry, Devi...",Pumpkin Soup,408.125000,1273.780,39.33400,111.29760,107.31420,242.9905,72.48640,1663.584,39954.54900,8905.82360,2540.5365,12.37140,88.51040
4,"Red Rice with Bathala Curry, Mustard Greens Cu...",String Hoppers with Pol Sambol,Coconut Roti with Dhal Curry,304.678000,1378.880,57.99280,93.03370,202.32750,308.9966,99.57520,2477.070,60123.21380,7188.46100,3299.2409,13.44240,172.60020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,Fish Kottu with Devilled Chicken,"Egg Fried Rice with Ala Badun, Vegetable Cutlets","Red Rice with Chicken Curry, Cashew Curry, Tro...",909.971000,1734.480,73.59915,82.75265,212.15410,463.6570,204.72410,3102.770,27029.65475,20864.31980,7472.8970,14.85395,182.24390
199996,"Red Rice with Parippu, Tomato Curry, Vegetable...","Chicken Fried Rice with Ala Badun, Cashew Curry",Coconut Roti with Fish Ambul Thiyal,1082.542000,1775.250,70.30400,116.97850,192.51750,500.0985,138.20800,3251.318,46516.95900,21770.17700,8176.2990,11.33950,165.67700
199997,Hoppers,"Red Rice with Chicken Curry, Beetroot curry, K...","Vegetable Fried Rice with Wambatu Moju, Cashew...",611.507857,1755.605,53.40785,117.71200,223.42295,405.8020,121.73010,2307.994,52973.73200,20356.11515,6584.9755,18.49190,172.29110
199998,Gotu Kola Kanda,"Red Rice with Pineapple Curry, Beetroot curry,...","Egg Fried Rice with Ala Badun, Fish Cutlets",483.850000,1384.290,43.96545,91.19495,164.94830,337.0390,213.43830,2599.690,35868.61975,7705.21390,3619.7880,14.26885,140.10920


In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import pickle
from sklearn.model_selection import train_test_split

weights = {
    'Price': 2.5,
    'Total Energy(Kcal)': 2,
    'Total Protein(g)': 2.2,
    'Total fat(g)': 0.5,
    'Total Carbohydrates(g)': 0.5,
    'Total Magnesium(mg)': 0.05,
    'Total Sodium(mg)': 0.05,
    'Total Potassium(mg)': 0.05,
    'Total Saturated Fatty Acids(mg)': 0.2,
    'Total Monounsaturated Fatty Acids(mg)': 0.05,
    'Total Polyunsaturated Fatty Acids(mg)': 0.1,
    'Total Free sugar(g)': 0.5,
    'Total Starch(g)': 0.1
}


def apply_weights(X, columns = []):
    X = pd.DataFrame(X, columns=columns)
    for feature, weight in weights.items():
        if feature in columns:
            X[feature] *= weight
    return X

def train_knn_model(
    n_neighbors=1,
    metric="cosine",
    algorithm="brute",
    model_filename="knn_model.pkl",
):
    """
    Trains a K-Nearest Neighbors model, saves it as a pickle file, and returns the pipeline.

    :param data_path: Path to the CSV file containing the data
    :param n_neighbors: Number of neighbors to use for KNN
    :param metric: Distance metric to use (default is 'cosine')
    :param algorithm: Algorithm to compute the nearest neighbors (default is 'brute')
    :param model_filename: Name of the file where the model will be saved
    :return: Trained pipeline object
    """

    # Load the data
    df_final = pd.read_csv('../datasets/meals/FinalPermutations.csv')
    columns = list(weights.keys())

    df_final["Meal_Plan"] = df_final.apply(lambda x : f'{x["Breakfast"]} / {x["Lunch"]} / {x["Dinner"]}', axis=1)

    X = df_final[columns]
    y = df_final['Meal_Plan']

    scaler = StandardScaler()
    prep_data = scaler.fit_transform(df_final[columns].to_numpy())
    prep_data = apply_weights(prep_data, columns=columns)

    neigh = NearestNeighbors(metric=metric, algorithm=algorithm, n_neighbors=n_neighbors)
    neigh.fit(prep_data)
    
    nn_transformer = FunctionTransformer(neigh.kneighbors, kw_args={"return_distance": False})
    weight_transformer = FunctionTransformer(apply_weights, validate=False, kw_args={"columns": columns})

    pipeline = Pipeline(steps=[
        ('std_scaler', scaler),
        ('weighting', weight_transformer ),
        ('NN', nn_transformer)
    ])
    print("pipeline created successfully... ")

    # Save the pipeline to a pickle file
    with open("../models/" + model_filename, "wb") as f:
        pickle.dump(pipeline, f)
    print("model saved successfully --------------------- ")

    return pipeline

train_knn_model(n_neighbors = 1)

pipeline created successfully... 
model saved successfully --------------------- 


Pipeline(steps=[('std_scaler', StandardScaler()),
                ('weighting',
                 FunctionTransformer(func=<function apply_weights at 0x120f576a0>,
                                     kw_args={'columns': ['Price',
                                                          'Total Energy(Kcal)',
                                                          'Total Protein(g)',
                                                          'Total fat(g)',
                                                          'Total '
                                                          'Carbohydrates(g)',
                                                          'Total Magnesium(mg)',
                                                          'Total Sodium(mg)',
                                                          'Total Potassium(mg)',
                                                          'Total Saturated '
                                                          'Fatty Acids(mg)',
                                                          'Total '
                                                          'Monounsaturated '
                                                          'Fatty Acids(mg)',
                                                          'Total '
                                                          'Polyunsaturated '
                                                          'Fatty Acids(mg)',
                                                          'Total Free sugar(g)',
                                                          'Total Starch(g)']})),
                ('NN',
                 FunctionTransformer(func=<bound method KNeighborsMixin.kneighbors of NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=1)>,
                                     kw_args={'return_distance': False}))])

In [12]:
with open('../models/knn_model.pkl', "rb") as f:
    pipeline = pickle.load(f)


# p = preference_permutations_df.iloc[5:7][columns].to_numpy().reshape(2,13)

pipeline.transform([
    [2.85590000e+02,
     6.27155000e+03, 
     4.47925000e+01, 
     2.21193000e+02,
        1.48704000e+02, 
        3.35159500e+02, 
        1.70980500e+02, 
        2.91038000e+03,
        7.16907590e+04, 
        9.04927400e+03, 
        3.82884100e+03, 
        2.52750000e+01,
        1.10202000e+02
    ]
])


array([[50483]])